In [1]:
cd /kaggle/input/c4w3-files

/kaggle/input/c4w3-files


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import time
import json
import torch
import string
import textwrap
import traceback
import itertools
import numpy as np 
import torch.nn as nn
from termcolor import colored

import transformer_utils_PT
import utils_PT

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
wrapper = textwrap.TextWrapper(width=70)
torch.manual_seed(2024)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session\

/kaggle/input/c4w3-files/c4-en-10k.json
/kaggle/input/c4w3-files/sentencepiece.model
/kaggle/input/c4w3-files/train-v2.0.json
/kaggle/input/c4w3-files/transformer_utils_PT.py
/kaggle/input/c4w3-files/inputs_targets_pairs_file.txt
/kaggle/input/c4w3-files/utils_PT.py
/kaggle/input/c4w3-files/c4-en-10k.jsonl
/kaggle/input/c4w3-files/data.txt


# Prepare the data for pretraining T5

## C4 dataset

In [3]:
with open('c4-en-10k.jsonl', 'r') as file:
    example_jsons = [json.loads(line.strip()) for line in file]
    
for i in range(5):
    print(f"example number {i+1}: \n\n{example_jsons[i]}\n")

example number 1: 

{'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.'}

example number 2: 

{'text': 'Discussion in \'Mac OS X Lion (10.7)\' started by axboi87, Jan 20, 2012.\nI\'ve got a 500gb internal drive and a 240gb SSD.\nWhen trying to restore using disk utility i\'m given the erro

## Process C4


In [4]:
natural_language_texts = [example_json['text'] for example_json in example_jsons]

print(natural_language_texts[0])

Beginners BBQ Class Taking Place in Missoula!
Do you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.
He will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.
The cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.


## Decode to Natural Language

In [25]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('sentencepiece.model')

In [74]:
tokenized_text = [(list(tokenizer.encode(word, add_special_tokens=False)), word) for word in natural_language_texts[2].split()]

print("Word\t\t-->\tTokenization")
print("-"*40)
for element in tokenized_text:
    print(f"{element[1]:<8}\t-->\t{element[0]}")

Word		-->	Tokenization
----------------------------------------
Foil    	-->	[4452, 173]
plaid   	-->	[30772]
lycra   	-->	[3, 120, 2935]
and     	-->	[11]
spandex 	-->	[8438, 26, 994]
shortall	-->	[710, 1748]
with    	-->	[28]
metallic	-->	[18813]
slinky  	-->	[3, 7, 4907, 63]
insets. 	-->	[16, 2244, 7, 5]
Attached	-->	[28416, 15, 26]
metallic	-->	[18813]
elastic 	-->	[15855]
belt    	-->	[6782]
with    	-->	[28]
O-ring. 	-->	[411, 18, 1007, 5]
Headband	-->	[3642, 3348]
included.	-->	[1285, 5]
Great   	-->	[1651]
hip     	-->	[5436]
hop     	-->	[13652]
or      	-->	[42]
jazz    	-->	[9948]
dance   	-->	[2595]
costume.	-->	[11594, 5]
Made    	-->	[6465]
in      	-->	[16]
the     	-->	[8]
USA.    	-->	[2312, 5]


In [101]:
vocab = tokenizer.get_vocab()

last_10 = sorted(vocab.items(), key=lambda x: x[1])[-100:]
last_10

[('<extra_id_99>', 32000),
 ('<extra_id_98>', 32001),
 ('<extra_id_97>', 32002),
 ('<extra_id_96>', 32003),
 ('<extra_id_95>', 32004),
 ('<extra_id_94>', 32005),
 ('<extra_id_93>', 32006),
 ('<extra_id_92>', 32007),
 ('<extra_id_91>', 32008),
 ('<extra_id_90>', 32009),
 ('<extra_id_89>', 32010),
 ('<extra_id_88>', 32011),
 ('<extra_id_87>', 32012),
 ('<extra_id_86>', 32013),
 ('<extra_id_85>', 32014),
 ('<extra_id_84>', 32015),
 ('<extra_id_83>', 32016),
 ('<extra_id_82>', 32017),
 ('<extra_id_81>', 32018),
 ('<extra_id_80>', 32019),
 ('<extra_id_79>', 32020),
 ('<extra_id_78>', 32021),
 ('<extra_id_77>', 32022),
 ('<extra_id_76>', 32023),
 ('<extra_id_75>', 32024),
 ('<extra_id_74>', 32025),
 ('<extra_id_73>', 32026),
 ('<extra_id_72>', 32027),
 ('<extra_id_71>', 32028),
 ('<extra_id_70>', 32029),
 ('<extra_id_69>', 32030),
 ('<extra_id_68>', 32031),
 ('<extra_id_67>', 32032),
 ('<extra_id_66>', 32033),
 ('<extra_id_65>', 32034),
 ('<extra_id_64>', 32035),
 ('<extra_id_63>', 32036),
 

In [75]:
print(f"tokenized: {tokenizer.encode('Beginners', add_special_tokens=False)}\ndetokenized: {tokenizer.decode(tokenizer.encode('Beginners', add_special_tokens=False))}")

tokenized: [12847, 277]
detokenized: Beginners
